## Import Libraries

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

## Helper Functions

### Column Namer

In [ ]:


def columnNamer(cols, isBigFive, isHultDna):
    '''
    This function renames columns depending if they are from the big 5 or 
    from Hult DNA. The format of the columns is <B5 or DNA>_Q<n>__<NAME_OF_COLUMN>.
    
    ----------------------
    Params
    ----------------------
    
    cols: Column name list.
    isBigFive: Boolean. If True, function will assume that all the columns are from the big five. 
    isHuldDNA: Boolean. If True, function will assume that all the columns are from the hult dna.
    
    '''
    

    cols = cols.str.lower()
    cols = cols.str.replace(' ', '_') # Fill spaces with underscore 
    cols = cols.str.replace("'", '') # Remove quotes from questions


    if isBigFive == True:
        _cols =  []
        for i in range(len(cols)):
            _cols.append(f'B5_Q{i+1}__{cols[i]}')
        return _cols

    elif isHultDna == True: 

        _cols =  []

        for i in range(len(cols)):
            _cols.append(f'DNA_Q{i+1}__{cols[i]}')
        return _cols 

    else:
        print('Call Manwe, Morgoth introduced a bug in this function')

## Domain Knowledge Research


### Nemid & Pastva (2013)

 - Big Five personality traits did not differentiate between Mac and PC owners. Students overall rated Macs higher on various product attributes (attractive style, cool, youthful, and exciting) and PCs higher on reasonable price and good for gaming.
 
 - PC owners placed greater importance on cost as a determinant of brand choice, whereas Mac owners placed greater emphasis on style. 
 
 - Personality traits may have more nuanced effects on brand choices, as shown by relationships between Neuroticism and greater importance placed on cost and lesser importance placed on ease of use. 
     - **Personality Traits are more important in the brand choice!**
     - More neuroticism, more importance in cost and less importance in ease of use
     
- Openness to Experience was associated with greater importance placed on reliability and lesser importance placed on style.
    - **More openness to experience, more importance in reliability and less importance in style**
    
    
https://www.researchgate.net/publication/259540094_I'm_a_Mac_versus_I'm_a_PC_Personality_Differences_between_Mac_and_PC_Users_in_a_College_Sample

<br> 

___________

<br>

### PC World

<br>


- People who purchase Macs fall into what the branding company calls the "Openness 5" personality category -- which means they are more liberal, less modest and more assured of their own superiority than the population at large.
-  People from Openness 5 seek rich, varied and novel experiences, according to the company, and believe that imagination and intellectual curiosity are as important to life as more rational or pragmatic endeavors.
    - Hypothesis: Mac users will have more opennes to new adventures. 

## Data Exploration

### General Data Set Exploration

- 1 missing value in ethnicity
- 1 participant said MAC instead of Macbook

In [ ]:
# Read Data
original_df = pd.read_excel('Survey_Data_Final_Exam.xlsx')

original_df.loc[:,'What laptop do you currently have?'].value_counts()

### Dividing Columns Big5 vs Hult DNA

In [ ]:


big_five = original_df.iloc[:, 1:51] # Subset big five-related columns
huld_dna = original_df.iloc[:,52:72] # Subset Hult DNA



# Change Column Names
big_five.columns = columnNamer(big_five.columns,
            isBigFive=True, 
            isHultDna=False)

huld_dna.columns = columnNamer(huld_dna.columns,
            isBigFive=False,
           isHultDna=True)

## Modelling


### PCA

In [ ]:

pca = PCA(n_components=5)

pca_fit = pca.fit_transform(big_five)

factor_loadings = pd.DataFrame(np.transpose(pca.components_))
factor_loadings.set_index(big_five.columns)
